In [1]:
!pip install graphviz

In [6]:
import time
import math

# ==============================
# Board & global counters
# ==============================
board = [' ' for _ in range(9)]
minimax_nodes_visited = 0
alpha_beta_nodes_visited = 0

# ==============================
# Helpers
# ==============================
def print_board(board_state):
    """Prints the Tic-Tac-Toe board and index helper."""
    print("\nCurrent board:")
    for i in range(3):
        print(f" {board_state[i*3]} | {board_state[i*3+1]} | {board_state[i*3+2]} ")
        if i < 2:
            print("---|---|---")
    # Show indices for empty cells to help humans choose valid moves
    empties = [str(i) for i, c in enumerate(board_state) if c == ' ']
    if empties:
        print(f"\nEmpty positions: {', '.join(empties)}\n")
    else:
        print()

def check_win(board_state, player):
    """Checks if 'player' has won."""
    win_lines = [
        [0,1,2],[3,4,5],[6,7,8],  # rows
        [0,3,6],[1,4,7],[2,5,8],  # cols
        [0,4,8],[2,4,6]           # diags
    ]
    return any(all(board_state[i] == player for i in line) for line in win_lines)

def check_draw(board_state):
    """Draw ⇔ board full and no winner."""
    return ' ' not in board_state and not check_win(board_state, 'X') and not check_win(board_state, 'O')

def get_empty_cells(board_state):
    """Indices of empty cells."""
    return [i for i, cell in enumerate(board_state) if cell == ' ']

# ==============================
# Minimax
# ==============================
def minimax(board_state, depth, is_maximizing):
    """
    Classic Minimax.
    Scoring: O (AI) aims to maximize (10 - depth); X (human) aims to minimize (depth - 10).
    """
    global minimax_nodes_visited
    minimax_nodes_visited += 1

    # terminal checks
    if check_win(board_state, 'O'):
        return 10 - depth
    if check_win(board_state, 'X'):
        return depth - 10
    if check_draw(board_state):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for mv in get_empty_cells(board_state):
            board_state[mv] = 'O'
            score = minimax(board_state, depth + 1, False)
            board_state[mv] = ' '
            best_score = max(best_score, score)
        return best_score
    else:
        best_score = math.inf
        for mv in get_empty_cells(board_state):
            board_state[mv] = 'X'
            score = minimax(board_state, depth + 1, True)
            board_state[mv] = ' '
            best_score = min(best_score, score)
        return best_score

def find_best_move_minimax(board_state):
    best_move, best_score = -1, -math.inf
    for mv in get_empty_cells(board_state):
        board_state[mv] = 'O'
        score = minimax(board_state, 0, False)
        board_state[mv] = ' '
        if score > best_score:
            best_score, best_move = score, mv
    return best_move

# ==============================
# Alpha–Beta Pruning
# ==============================
def alpha_beta_pruning(board_state, depth, alpha, beta, is_maximizing):
    """
    Alpha–Beta pruning for Tic-Tac-Toe.
    O (AI) is maximizing, X (human) is minimizing.
    """
    global alpha_beta_nodes_visited
    alpha_beta_nodes_visited += 1

    # --- terminal checks identical to minimax ---
    if check_win(board_state, 'O'):
        return 10 - depth
    if check_win(board_state, 'X'):
        return depth - 10
    if check_draw(board_state):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for mv in get_empty_cells(board_state):
            board_state[mv] = 'O'
            score = alpha_beta_pruning(board_state, depth + 1, alpha, beta, False)
            board_state[mv] = ' '
            best_score = max(best_score, score)
            alpha = max(alpha, best_score)
            if beta <= alpha:   # prune
                break
        return best_score
    else:
        best_score = math.inf
        for mv in get_empty_cells(board_state):
            board_state[mv] = 'X'
            score = alpha_beta_pruning(board_state, depth + 1, alpha, beta, True)
            board_state[mv] = ' '
            best_score = min(best_score, score)
            beta = min(beta, best_score)
            if beta <= alpha:   # prune
                break
        return best_score

def find_best_move_alpha_beta(board_state):
    best_move, best_score = -1, -math.inf
    for mv in get_empty_cells(board_state):
        board_state[mv] = 'O'
        score = alpha_beta_pruning(board_state, 0, -math.inf, math.inf, False)
        board_state[mv] = ' '
        if score > best_score:
            best_score, best_move = score, mv
    return best_move

# ==============================
# Gameplay & comparison
# ==============================
def play_game():
    """Interactive loop; lets you choose algorithm and play."""
    print("Tic-Tac-Toe AI")
    print("You are 'X'; AI is 'O'.")
    print("Choose AI algorithm:\n  1) Minimax\n  2) Alpha–Beta")

    # get choice
    while True:
        try:
            choice = int(input("Enter your choice (1 or 2): ").strip())
            if choice in (1, 2): break
            print("Invalid choice. Enter 1 or 2.")
        except ValueError:
            print("Please enter a number (1 or 2).")

    global board, minimax_nodes_visited, alpha_beta_nodes_visited
    board = [' ' for _ in range(9)]
    minimax_nodes_visited = 0
    alpha_beta_nodes_visited = 0
    total_ai_time = 0.0

    while True:
        print_board(board)

        # human turn
        if not get_empty_cells(board):
            break
        while True:
            try:
                mv = int(input("Your move (0-8): ").strip())
                if 0 <= mv <= 8 and board[mv] == ' ':
                    board[mv] = 'X'
                    break
                print("Invalid move. Choose an empty index 0–8.")
            except ValueError:
                print("Please enter a number 0–8.")

        if check_win(board, 'X'):
            print_board(board)
            print("You win!")
            break
        if check_draw(board):
            print_board(board)
            print("Draw.")
            break

        # AI turn
        print("AI is thinking...")
        start = time.perf_counter()
        if choice == 1:
            ai_mv = find_best_move_minimax(board)
        else:
            ai_mv = find_best_move_alpha_beta(board)
        total_ai_time += (time.perf_counter() - start)

        if ai_mv != -1:
            board[ai_mv] = 'O'

        if check_win(board, 'O'):
            print_board(board)
            print("AI wins.")
            break
        if check_draw(board):
            print_board(board)
            print("Draw.")
            break

    # report
    print("\n--- PERFORMANCE ---")
    if choice == 1:
        print("Algorithm: Minimax")
        print(f"Nodes visited: {minimax_nodes_visited}")
    else:
        print("Algorithm: Alpha–Beta")
        print(f"Nodes visited: {alpha_beta_nodes_visited}")
    print(f"Total AI thinking time: {total_ai_time:.6f} s")

if __name__ == "__main__":
    play_game()


Tic-Tac-Toe AI
You are 'X'; AI is 'O'.
Choose AI algorithm:
  1) Minimax
  2) Alpha–Beta
Enter your choice (1 or 2): 2

Current board:
   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Empty positions: 0, 1, 2, 3, 4, 5, 6, 7, 8

Your move (0-8): 3
AI is thinking...

Current board:
 O |   |   
---|---|---
 X |   |   
---|---|---
   |   |   

Empty positions: 1, 2, 4, 5, 6, 7, 8

Your move (0-8): 4
AI is thinking...

Current board:
 O |   |   
---|---|---
 X | X | O 
---|---|---
   |   |   

Empty positions: 1, 2, 6, 7, 8

Your move (0-8): 6
AI is thinking...

Current board:
 O |   | O 
---|---|---
 X | X | O 
---|---|---
 X |   |   

Empty positions: 1, 7, 8

Your move (0-8): 1
AI is thinking...

Current board:
 O | X | O 
---|---|---
 X | X | O 
---|---|---
 X |   | O 

Empty positions: 7

AI wins.

--- PERFORMANCE ---
Algorithm: Alpha–Beta
Nodes visited: 8987
Total AI thinking time: 0.106850 s
